In [11]:
# pip install duckdb

In [12]:
import duckdb 
import os
import pandas as pd
import numpy as np
from pathlib import Path


con = duckdb.connect()

base_dir = Path("C:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis\\Dataset")
data_dir = str(base_dir).replace("\\","/")

In [13]:
data_dir

'C:/Users/henry/OneDrive/Personal Career/Personal Projects/GitHub/Revenue-Sustainability-Analysis/Dataset'

In [14]:
df = con.execute(f"""
    SELECT *
    FROM '{base_dir}/subscriptions.parquet'
                 """).df()

In [18]:
df

,subscription_id,customer_id,month,monthly_fee
0,None,1020,2024-10-01,200
1,S-1020-202411,1020,2024-11-01,<NA>
2,None,<NA>,2024-12-01,200
3,None,1020,2025-01-01,200
4,None,1020,2025-02-01,200
...,...,...,...,...
983,None,1988,2025-01-01,<NA>
984,None,1988,2025-02-01,500
985,None,1988,2025-03-01,500
986,S-1999-202504,1999,2025-04-01,500


In [16]:
import duckdb

con = duckdb.connect()

query = f"""
SELECT 'revenue' AS table_name,
       MIN(month) AS min_date,
       MAX(month) AS max_date
FROM '{base_dir}/revenue.parquet'

UNION ALL
SELECT 'events',
       MIN(occurred_at),
       MAX(occurred_at)
FROM '{base_dir}/events.parquet'

UNION ALL
SELECT 'subscriptions',
       MIN(month),
       MAX(month)
FROM '{base_dir}/subscriptions.parquet';
"""

con.execute(query).df()


,table_name,min_date,max_date
0,revenue,2024-01-01 00:00:00,2025-11-01 00:00:00
1,events,2025-05-20 18:51:40,2025-11-30 06:46:56
2,subscriptions,2024-01-01 00:00:00,2025-11-01 00:00:00
